In [1]:
import pandas as pd

### Function that merges all months into one DF

In [19]:
def merge_csv(path, files):

    final_df = pd.DataFrame()
    for file in files:
        name = f'{path}{file}'
        print(name)
        df = pd.read_csv(name)
        final_df = pd.concat([final_df, df], ignore_index=True)

    return final_df

In [33]:

files = files = ['april_2024.csv', 'may_2024.csv', 'july_2024.csv', 'august_2024.csv', 'july_2024.csv', 'september_2024.csv', 'october_2024.csv']
path = '/Users/rdhir/Documents/roni_dashboard_td2024/Data/'

df = merge_csv(path, files)

/Users/rdhir/Documents/roni_dashboard_td2024/Data/april_2024.csv
/Users/rdhir/Documents/roni_dashboard_td2024/Data/may_2024.csv
/Users/rdhir/Documents/roni_dashboard_td2024/Data/july_2024.csv
/Users/rdhir/Documents/roni_dashboard_td2024/Data/august_2024.csv
/Users/rdhir/Documents/roni_dashboard_td2024/Data/july_2024.csv
/Users/rdhir/Documents/roni_dashboard_td2024/Data/september_2024.csv
/Users/rdhir/Documents/roni_dashboard_td2024/Data/october_2024.csv


In [38]:
df.head(50)

,Order #,Sent Date,Modifier,Option Group Name,Parent Menu Selection,Order ID
0,18,2024-04-13 11:34:00,Regular,Noods,Mac and Cheese,13-18
1,18,2024-04-13 11:34:00,Cheddar,Choose Your Cheese,Mac and Cheese,13-18
2,18,2024-04-13 11:34:00,Pulled Pork,Choose Your Meats,Mac and Cheese,13-18
3,18,2024-04-13 11:34:00,Broccoli,Choose Your Toppings,Mac and Cheese,13-18
4,18,2024-04-13 11:34:00,Tomatoes,Choose Your Toppings,Mac and Cheese,13-18
5,18,2024-04-13 11:34:00,Breadcrumbs,Choose Your Toppings,Mac and Cheese,13-18
6,18,2024-04-13 11:34:00,BBQ,Choose Your Drizzles,Mac and Cheese,13-18
7,18,2024-04-13 11:34:00,No Side,Choose Your Side,Mac and Cheese,13-18
8,18,2024-04-13 11:34:00,Water,Choose Your Drink,Mac and Cheese,13-18
9,18,2024-04-13 11:34:00,Regular,Noods,Mac and Cheese,13-18


In [40]:
## convert toppings to dummy variables
encoded_df = pd.get_dummies(df, columns=['Modifier'])
encoded_df.head(10)

,Order #,Sent Date,Option Group Name,Parent Menu Selection,Order ID,Modifier_ can I get extra corn bell pepper and bread cr,Modifier_ extra extra bell peppers extra extra corn extra,Modifier_(Please label for Aidan),Modifier_2X Shirt,Modifier_Add alfredo and pepper jack cheese as well!,...,Modifier_not to toasted please :),Modifier_not too heavy on the sauces,Modifier_only a light drizzle of garlic Parmesan,Modifier_pesto on the side please,Modifier_please add all the cheeses,Modifier_please add extra pineapples (for dine in),Modifier_please mark each meal so we can tell whose it is,Modifier_plz hella broccoli,Modifier_sauce on side,Modifier_xtra sauce/cheese please! thanks!
0,18,2024-04-13 11:34:00,Noods,Mac and Cheese,13-18,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,18,2024-04-13 11:34:00,Choose Your Cheese,Mac and Cheese,13-18,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,18,2024-04-13 11:34:00,Choose Your Meats,Mac and Cheese,13-18,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,18,2024-04-13 11:34:00,Choose Your Toppings,Mac and Cheese,13-18,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,18,2024-04-13 11:34:00,Choose Your Toppings,Mac and Cheese,13-18,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,18,2024-04-13 11:34:00,Choose Your Toppings,Mac and Cheese,13-18,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,18,2024-04-13 11:34:00,Choose Your Drizzles,Mac and Cheese,13-18,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,18,2024-04-13 11:34:00,Choose Your Side,Mac and Cheese,13-18,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,18,2024-04-13 11:34:00,Choose Your Drink,Mac and Cheese,13-18,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,18,2024-04-13 11:34:00,Noods,Mac and Cheese,13-18,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [41]:
df['Modifier'].unique()

array(['Regular', 'Cheddar', 'Pulled Pork', 'Broccoli', 'Tomatoes',
       'Breadcrumbs', 'BBQ', 'No Side', 'Water', 'Pepper Jack', 'Brisket',
       'Corn', 'Mushrooms', 'Parmesan', 'No Drink', 'Alfredo',
       'Jalapenos', 'Unlimited Fountain Drinks', 'No Toppings',
       'Cheesy Broccoli', 'Cheesy Garlic Bread', 'No Meat', 'No Drizzle',
       'Pepperjack', 'Bacon', 'Garlic Parmesan', 'Grilled Chicken',
       'Bell Peppers', 'Hot Honey', 'Garlic Bread', 'Ranch', 'Cheesecake',
       'Buffalo', 'Ham', 'Melted Cheddar', 'No Mac', 'Pesto', 'Sprite',
       'Coke', 'Pepper Jack Mac', 'Pineapple', 'Melted Pepper Jack',
       'MIX', 'Mix PJ And Cheddar', 'GLUTEN FREE', 'Melted Parmesan',
       'Cheddar Mac', 'Alfredo Mac', 'Side Mac', 'Diet Coke', 'Onions',
       'Onion', 'All Cheese!!!', 'I DO NOT NEED UTENSILS (Save waste!)',
       'O J And BP On Side', 'Bacon On Side', 'Hot Honey On Half',
       'MIXED Melted Cheese', 'Bbq On Side',
       'Gluten-Free (ask store for safe toppi

In [42]:
df['Parent Menu Selection'].unique()

array(['Mac and Cheese', 'Grilled Cheese Sandwich', 'Sides/Desserts',
       'Drinks', 'MIX',
       'Mac and Cheese Party Tray (Plus FREE Garlic Bread)',
       'Garlic Bread (Party Size)', nan], dtype=object)

In [48]:
df[df['Parent Menu Selection'] == 'MIX'].head(50)

,Order #,Sent Date,Modifier,Option Group Name,Parent Menu Selection,Order ID
2038,11,2024-04-14 12:16:00,Cheddar,Mix Bases,MIX,14-11
2039,11,2024-04-14 12:16:00,Alfredo,Mix Bases,MIX,14-11
2049,11,2024-04-14 12:16:00,Cheddar,Mix Bases,MIX,14-11
2050,11,2024-04-14 12:16:00,Alfredo,Mix Bases,MIX,14-11
2437,43,2024-04-14 14:14:00,Cheddar,Mix Bases,MIX,14-43
2438,43,2024-04-14 14:14:00,Pepper Jack,Mix Bases,MIX,14-43
3050,111,2024-04-14 16:32:00,Cheddar,Mix Bases,MIX,14-111
3051,111,2024-04-14 16:32:00,Pepper Jack,Mix Bases,MIX,14-111
8253,84,2024-04-17 16:20:00,Cheddar,Mix Bases,MIX,17-84
8254,84,2024-04-17 16:20:00,Pepper Jack,Mix Bases,MIX,17-84


In [49]:
df[df['Order #'] == 11]

,Order #,Sent Date,Modifier,Option Group Name,Parent Menu Selection,Order ID
2037,11,2024-04-14 12:16:00,Regular,Noods,Mac and Cheese,14-11
2038,11,2024-04-14 12:16:00,Cheddar,Mix Bases,MIX,14-11
2039,11,2024-04-14 12:16:00,Alfredo,Mix Bases,MIX,14-11
2040,11,2024-04-14 12:16:00,MIX,Choose Your Cheese,Mac and Cheese,14-11
2041,11,2024-04-14 12:16:00,Pulled Pork,Choose Your Meats,Mac and Cheese,14-11
...,...,...,...,...,...,...
276229,11,2024-10-30 11:27:00,Broccoli,Choose Your Toppings,Mac and Cheese,30-11
276230,11,2024-10-30 11:27:00,Corn,Choose Your Toppings,Mac and Cheese,30-11
276231,11,2024-10-30 11:27:00,BBQ,Choose Your Drizzles,Mac and Cheese,30-11
276232,11,2024-10-30 11:27:00,No Side,Choose Your Side,Mac and Cheese,30-11


In [51]:
# Create a dish counter to differentiate each distinct configuration in the same order
df["Modifier"] = df["Modifier"].fillna("").astype(str)
df["Dish Number"] = (df["Option Group Name"] == "Choose Your Drink").cumsum()

# Pivot table to separate each dish configuration
df_pivot = (
    df.groupby(["Order ID", "Dish Number", "Parent Menu Selection", "Sent Date", "Order #", "Option Group Name"])["Modifier"]
    .apply(", ".join)  # Aggregate modifiers within the same option group
    .unstack()         # Convert Option Group Name into columns
    .reset_index()     # Reset index to make it a flat DataFrame
)

In [54]:
df_pivot[df_pivot['Parent Menu Selection'] == 'Grilled Cheese Sandwich']

Option Group Name,Order ID,Dish Number,Parent Menu Selection,Sent Date,Order #,Choose Your Cheese,Choose Your Drink,Choose Your Drizzles,Choose Your Meats,Choose Your Melted Cheese,Choose Your Shirt,Choose Your Side,Choose Your Toppings,Do you want Mac and Cheese added inside?,Mac and Cheese Options,Mix Bases,Noods,Sides,Utensils?
12,1-10,10629,Grilled Cheese Sandwich,2024-07-01 11:37:00,10,NaN,NaN,BBQ,No Meat,Melted Pepper Jack,NaN,No Side,"Broccoli, Corn, Jalapenos, Tomatoes, Bell Pepp...",No Mac,NaN,NaN,NaN,NaN,NaN
13,1-10,10630,Grilled Cheese Sandwich,2024-07-01 11:37:00,10,NaN,No Drink,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,1-10,17703,Grilled Cheese Sandwich,2024-07-01 11:37:00,10,NaN,NaN,BBQ,No Meat,Melted Pepper Jack,NaN,No Side,"Broccoli, Corn, Jalapenos, Tomatoes, Bell Pepp...",No Mac,NaN,NaN,NaN,NaN,NaN
18,1-10,17704,Grilled Cheese Sandwich,2024-07-01 11:37:00,10,NaN,No Drink,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,1-100,20641,Grilled Cheese Sandwich,2024-09-01 16:53:00,100,NaN,NaN,Hot Honey,Pulled Pork,Melted Pepper Jack,NaN,No Side,Jalapenos,Pepper Jack Mac,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58827,9-90,26472,Grilled Cheese Sandwich,2024-10-09 19:04:00,90,NaN,No Drink,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58839,9-93,8484,Grilled Cheese Sandwich,2024-05-09 15:15:00,93,NaN,NaN,Garlic Parmesan,Brisket,Melted Cheddar,NaN,No Side,Parmesan,Pepper Jack Mac,NaN,NaN,NaN,NaN,NaN
58840,9-93,8485,Grilled Cheese Sandwich,2024-05-09 15:15:00,93,NaN,No Drink,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58870,9-98,21966,Grilled Cheese Sandwich,2024-09-09 16:02:00,98,NaN,NaN,BBQ,"Bacon, Ham",Melted Cheddar,NaN,Cheesy Garlic Bread,"Onions, Bell Peppers",Alfredo Mac,NaN,NaN,NaN,NaN,NaN
